Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### (1) L2 Regularization for Logistic Model

In [22]:
# Subset the training data for faster turnaround.
train_subset = 10000
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation (with L2 Regularization).
  logits = tf.matmul(tf_train_dataset, weights) + biases
  reg = tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) 
      + beta * reg)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [23]:
num_steps = 801

with tf.Session(graph=graph) as session:
  # One-time operation to initialize parameters.
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 45.459873
Training accuracy: 10.3%
Validation accuracy: 13.0%
Loss at step 100: 11.981088
Training accuracy: 73.8%
Validation accuracy: 72.6%
Loss at step 200: 4.531105
Training accuracy: 79.1%
Validation accuracy: 77.0%
Loss at step 300: 1.995046
Training accuracy: 82.5%
Validation accuracy: 79.7%
Loss at step 400: 1.133299
Training accuracy: 83.8%
Validation accuracy: 81.2%
Loss at step 500: 0.837357
Training accuracy: 84.6%
Validation accuracy: 81.8%
Loss at step 600: 0.734514
Training accuracy: 84.8%
Validation accuracy: 82.0%
Loss at step 700: 0.698444
Training accuracy: 84.8%
Validation accuracy: 82.2%
Loss at step 800: 0.685698
Training accuracy: 84.9%
Validation accuracy: 82.1%
Test accuracy: 88.8%


### (2) L2 Regularization for Neural Network

In [43]:
batch_size = 128
hidden_nodes_num = 1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_num]))
  biases = tf.Variable(tf.zeros([hidden_nodes_num]))
    
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_num, num_labels])
  )
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation (with L2 Regularization).
  logits = tf.matmul(hidden, weights_2) + biases_2
  reg = tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_2)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) 
      + beta * reg)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weights_2) + biases_2)

In [44]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3436.022461
Minibatch accuracy: 12.5%
Validation accuracy: 29.8%
Minibatch loss at step 500: 21.216160
Minibatch accuracy: 87.5%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 0.948032
Minibatch accuracy: 81.2%
Validation accuracy: 83.0%
Minibatch loss at step 1500: 0.580426
Minibatch accuracy: 86.7%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 0.603905
Minibatch accuracy: 91.4%
Validation accuracy: 83.3%
Minibatch loss at step 2500: 0.716036
Minibatch accuracy: 84.4%
Validation accuracy: 83.3%
Minibatch loss at step 3000: 0.766855
Minibatch accuracy: 83.6%
Validation accuracy: 83.3%
Test accuracy: 90.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [26]:
num_steps = 3001
dataset_size = 512

train_dataset_small = train_dataset[:dataset_size, :]
train_labels_small = train_labels[:dataset_size, :]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3503.620605
Minibatch accuracy: 10.2%
Validation accuracy: 31.8%
Minibatch loss at step 500: 20.990841
Minibatch accuracy: 100.0%
Validation accuracy: 76.9%
Minibatch loss at step 1000: 0.454689
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Minibatch loss at step 1500: 0.307458
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 2000: 0.262368
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 2500: 0.271220
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 3000: 0.283879
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Test accuracy: 85.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

### (1) Dropout on the Hidden Layer

In [67]:
batch_size = 128
hidden_nodes_num = 1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_num]))
  biases = tf.Variable(tf.zeros([hidden_nodes_num]))
    
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  keep_prob = tf.constant(0.5, tf.float32)
  hidden_dropout = tf.nn.dropout(hidden, keep_prob)
    
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_num, num_labels])
  )
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation (with L2 Regularization).
  logits = tf.matmul(hidden_dropout, weights_2) + biases_2
  reg = tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_2)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) 
      + beta * reg)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weights_2) + biases_2)

In [69]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3633.521973
Minibatch accuracy: 7.8%
Validation accuracy: 29.6%
Minibatch loss at step 500: 21.373871
Minibatch accuracy: 83.6%
Validation accuracy: 83.6%
Minibatch loss at step 1000: 1.060322
Minibatch accuracy: 81.2%
Validation accuracy: 82.6%
Minibatch loss at step 1500: 0.633293
Minibatch accuracy: 86.7%
Validation accuracy: 83.2%
Minibatch loss at step 2000: 0.698706
Minibatch accuracy: 88.3%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 0.799390
Minibatch accuracy: 82.0%
Validation accuracy: 83.1%
Minibatch loss at step 3000: 0.868232
Minibatch accuracy: 81.2%
Validation accuracy: 83.1%
Test accuracy: 89.9%


### (2) Extreme Overfitting Case

In [70]:
num_steps = 3001
dataset_size = 512

train_dataset_small = train_dataset[:dataset_size, :]
train_labels_small = train_labels[:dataset_size, :]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3684.719727
Minibatch accuracy: 14.1%
Validation accuracy: 33.3%
Minibatch loss at step 500: 21.129734
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Minibatch loss at step 1000: 0.486337
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 1500: 0.362081
Minibatch accuracy: 99.2%
Validation accuracy: 78.5%
Minibatch loss at step 2000: 0.283960
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 2500: 0.305229
Minibatch accuracy: 100.0%
Validation accuracy: 78.9%
Minibatch loss at step 3000: 0.319067
Minibatch accuracy: 99.2%
Validation accuracy: 78.4%
Test accuracy: 85.6%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [75]:
batch_size = 128
hidden_nodes_num = 1024
beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_num]))
  biases = tf.Variable(tf.zeros([hidden_nodes_num]))
    
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  keep_prob = tf.constant(0.5, tf.float32)
  hidden_dropout = tf.nn.dropout(hidden, keep_prob)
    
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_num, num_labels])
  )
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation (with L2 Regularization).
  logits = tf.matmul(hidden_dropout, weights_2) + biases_2
  reg = tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_2)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) 
      + beta * reg)
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weights_2) + biases_2)

In [76]:
num_steps = 10000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 765.950684
Minibatch accuracy: 9.4%
Validation accuracy: 35.7%
Minibatch loss at step 500: 214.733826
Minibatch accuracy: 74.2%
Validation accuracy: 79.7%
Minibatch loss at step 1000: 116.953667
Minibatch accuracy: 76.6%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 70.453682
Minibatch accuracy: 87.5%
Validation accuracy: 82.0%
Minibatch loss at step 2000: 41.493332
Minibatch accuracy: 85.2%
Validation accuracy: 83.4%
Minibatch loss at step 2500: 25.214663
Minibatch accuracy: 86.7%
Validation accuracy: 84.2%
Minibatch loss at step 3000: 15.530758
Minibatch accuracy: 82.8%
Validation accuracy: 84.9%
Minibatch loss at step 3500: 9.657352
Minibatch accuracy: 81.2%
Validation accuracy: 85.4%
Minibatch loss at step 4000: 6.061148
Minibatch accuracy: 86.7%
Validation accuracy: 85.6%
Minibatch loss at step 4500: 3.811671
Minibatch accuracy: 88.3%
Validation accuracy: 86.0%
Minibatch loss at step 5000: 2.580362
Minibatch accuracy: 87.5%
Validatio